In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astropy import wcs
from scipy.spatial import KDTree
from astropy.io import fits
from astropy.coordinates import SkyCoord
import regions
from regions import Regions
from regions import Region
from spectral_cube import SpectralCube
from astroquery.jplspec import JPLSpec
from astroquery.splatalogue import Splatalogue
from dust_extinction.averages import RL85_MWGC, RRP89_MWGC, B92_MWAvg, I05_MWAvg, CT06_MWLoc, CT06_MWGC, GCC09_MWAvg, F11_MWGC, G21_MWAvg, D22_MWAvg
from importlib import reload
from astropy.visualization import simple_norm

In [2]:
# Setting up region cutout to be used. Make sure it overlaps with the PPMAP file you are using. 
# I used a rectangular region here, but any region shape can be used.
pos = SkyCoord('17:46:20.6290029866', '-28:37:49.5114204513', unit=(u.hour, u.deg))
l = 113.8*u.arcsec
w = 3.3*u.arcmin
reg = regions.RectangleSkyRegion(pos, width=l, height=w)

## PPMAP

In [9]:
ppmap = '/orange/adamginsburg/galactic_plane_surveys/higal/PPMAP_Results/l000_results/l000_cdens.fits'
ppmap_err = '/orange/adamginsburg/galactic_plane_surveys/higal/PPMAP_Results/l000_results/l000_sigdiffcdens.fits'

In [10]:
# Constants
## distance to cloud
distance = 5*u.kpc
## mean mass per particle of H2
mpp = 2.8*u.Da

In [11]:
r0 = reg#[0]
hdul = fits.open(ppmap)
ww = wcs.WCS(hdul[0].header)
preg = r0.to_pixel(ww)
mask = preg.to_mask()
cutout = mask.cutout(hdul[0].data)
#pixelarea = 1*u.pixel
cden = (mask.multiply(hdul[0].data) * (10**20) * u.cm**(-2))
        #* u.Unit(hdul[0].header['BUNIT']))
u.Unit(hdul[0].header['BUNIT'])

Unit("1e+20 / cm2")

In [12]:
coldens_ppmap = np.nanmean(cden)
coldens_ppmap

<Quantity 2.44691751e+22 1 / cm2>

In [13]:
pix_area_2 = ww.celestial.proj_plane_pixel_area()
pix_area_cm_2 = (pix_area_2 * (distance)**2).to(u.parsec**2, equivalencies=u.dimensionless_angles()).to(u.cm**2)
mass_ppmap = (pix_area_cm_2 * cden * mpp).to(u.M_sun).sum()
mass_ppmap

<Quantity 15287.31118604 solMass>

In [14]:
# using the PPMAP column density error map
coldens_ppmap_err = mask.multiply(fits.open(ppmap_err)[0].data[5]).mean()* (10**20) * u.cm**(-2)#*u.Unit(hdul[0].header['BUNIT'])
coldens_ppmap_err 

<Quantity 4.14044193e+20 1 / cm2>

In [15]:
mass_ppmap_err = mass_ppmap * coldens_ppmap_err / coldens_ppmap
mass_ppmap_err

<Quantity 258.67739222 solMass>